In [ ]:
%pip install -U transformers bitsandbytes accelerate peft trl wandb python-dotenv datasets huggingface_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split
from datetime import datetime
import wandb
from google.colab import userdata

In [ ]:
from huggingface_hub import login
login(token=userdata.get('HUGGINGFACE_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Loading and processing the dataset

In [ ]:
splits = {'train': 'data/train-00000-of-00001-7b34565378f02992.parquet', 'val': 'data/val-00000-of-00001-d7338c59b5e5031f.parquet', 'test': 'data/test-00000-of-00001-c830a979da438bff.parquet'}
df_train = pd.read_parquet("hf://datasets/PrevenIA/spanish-suicide-intent/" + splits["train"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df_train.head()

,Text,Label,dataset,__index_level_0__
0,¿Por qué hacer que mejore? ¿Por qué la escuela...,1,Hackaton Somos NLP,5071
1,- Roma \n- Milán \n- Nápoles \n- Turín \n- Pal...,0,Hackaton Somos NLP,9391
2,¿Cuál es la noticia más importante en tu área ...,0,Hackaton Somos NLP,1752
3,Una semana de identificación izquierda más bie...,1,Hackaton Somos NLP,5363
4,Podría ser que no estudiaras de la manera corr...,0,Hackaton Somos NLP,107


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136136 entries, 0 to 136135
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Text               136136 non-null  object
 1   Label              136136 non-null  int64 
 2   dataset            136136 non-null  object
 3   __index_level_0__  136136 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 4.2+ MB


In [ ]:
df_train.Label.value_counts()

,count
Label,
0,75496
1,60640


In [ ]:
# Filtra los datos con Label igual a 1
df_label_1 = df_train[df_train['Label'] == 1]

# Filtra los datos con Label igual a 0
df_label_0 = df_train[df_train['Label'] == 0]

# Toma 10,000 muestras aleatorias de cada grupo
sampled_label_1 = df_label_1.sample(n=10000, random_state=42)
sampled_label_0 = df_label_0.sample(n=10000, random_state=42)

# Combina ambos DataFrames
sampled_df = pd.concat([sampled_label_1, sampled_label_0])

# Reordena las filas aleatoriamente

df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)
df.Label.value_counts()

,count
Label,
0,10000
1,10000


In [ ]:
# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]

# Define the prompt generation functions
def generate_prompt(data_point):
    return f"""
            Clasifica el texto con la etiquta "1" si hay ideación/comportamiento suicida y la etiqueta "0" en otro caso, retorna la respuesta como la correspondiente etiqueta.
texto: {data_point["Text"]}
etiqueta: {data_point["Label"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Clasifica el texto con la etiquta "1" si hay ideación/comportamiento suicida y la etiqueta "0" en otro caso, retorna la respuesta como la correspondiente etiqueta.
texto: {data_point["Text"]}
etiqueta: """.strip()

# Generate prompts for training and evaluation data
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'Label']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["Text"])

<ipython-input-10-9030f6d2d232>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
<ipython-input-10-9030f6d2d232>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)


In [ ]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

In [ ]:
train_data['text'][3]

'Clasifica el texto con la etiquta "1" si hay ideación/comportamiento suicida y la etiqueta "0" en otro caso, retorna la respuesta como la correspondiente etiqueta.\ntexto: Cada vez que pienso en cómo eran las cosas antes de que naciera me da nostalgia no tenía ninguna neurona que me dijera cómo soy responsable de todas las cosas malas que están mal en este mundo y puedo arreglarlo si me deshago de la enfermedad que soy.\netiqueta: 1'

## Loading the model and tokenizer ( in 4-bit quantization to save the GPU memory)

In [ ]:
base_model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

## tracking experiment with wandb

In [ ]:
# Crear diccionario con los parámetros para wandb con los valores especificados
wandb_config = {
    "lora_alpha": 16,
    "lora_dropout": 0.2,
    "r": 64,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "num_train_epochs": 2,
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps": 8,
    "gradient_checkpointing": True,
    "optim": "paged_adamw_32bit",
    "logging_steps": 1,
    "learning_rate": 2e-4,
    "weight_decay": 0.001,
    "fp16": True,
    "bf16": False,
    "max_grad_norm": 0.3,
    "warmup_ratio": 0.03,
    "lr_scheduler_type": "cosine",
    "eval_strategy": "steps",
    "eval_steps": 0.2,
    "max_seq_length": 512,
    "packing": False,
    "add_special_tokens": False,
    "append_concat_token": False
}

In [ ]:
today = datetime.today()
run_date = today.strftime("%d-%m-%Y_%H_%M")

wb_token = userdata.get('wandb')

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune llama-3.1-8b-it on suicide intent-spanish',
    job_type="training",
    name=f"experiment_fine_tune_llama_{run_date}",
      # Track hyperparameters and run metadata
    config=wandb_config,
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: felipeandres29 (felipeandres29-universidad-eafit). Use `wandb login --relogin` to force relogin


# Building the model

In [ ]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules

['v_proj', 'o_proj', 'gate_proj', 'k_proj', 'q_proj', 'down_proj', 'up_proj']

In [ ]:
output_dir="llama-3.1-fine-tuned-model"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=wandb_config["lora_dropout"],
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=wandb_config["num_train_epochs"],                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="wandb",                  # report metrics to w&b
    eval_strategy="steps",              # save checkpoint every epoch
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:327: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you pass

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[at

Step,Training Loss,Validation Loss
800,1.180000,1.381940
1600,1.278200,1.362064
2400,1.182000,1.362719
3200,0.731500,1.355175
4000,0.914600,1.352951


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=4000, training_loss=1.1866371455937623, metrics={'train_runtime': 18352.9128, 'train_samples_per_second': 1.744, 'train_steps_per_second': 0.218, 'total_flos': 1.4725031572502938e+17, 'train_loss': 1.1866371455937623, 'epoch': 2.0})

In [ ]:
# Save trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('llama-3.1-fine-tuned-model/tokenizer_config.json',
 'llama-3.1-fine-tuned-model/special_tokens_map.json',
 'llama-3.1-fine-tuned-model/tokenizer.json')

# Testing model after fine-tuning

In [ ]:
def evaluate(y_true, y_pred):
    labels = ["1", "0"]
    mapping = {label: idx for idx, label in enumerate(labels)}

    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data

    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)

    # Get classification report as a dictionary
    class_report_dict = classification_report(y_true, y_pred, target_names=labels, labels=list(range(len(labels))), output_dict=True)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

    return accuracy, class_report_dict, conf_matrix,labels

def predict(test, model, tokenizer):
    y_pred = []
    categories = ["1", "0"]

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["Text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=2,
                        temperature=0.1)

        result = pipe(prompt)
        answer = result[0]['generated_text'].split("etiqueta:")[-1].strip()

        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")

    return y_pred

In [ ]:
y_pred = predict(X_test, model, tokenizer)

100%|██████████| 2000/2000 [10:34<00:00,  3.15it/s]


In [ ]:
y_true = y_true.astype(str)

In [ ]:
accuracy, class_report, conf_matrix, labels = evaluate(y_true, y_pred)

Accuracy: 0.924
Accuracy for label 1: 0.932
Accuracy for label 0: 0.916

Classification Report:
              precision    recall  f1-score   support

           1       0.92      0.93      0.93      1006
           0       0.93      0.92      0.92       994

    accuracy                           0.92      2000
   macro avg       0.92      0.92      0.92      2000
weighted avg       0.92      0.92      0.92      2000


Confusion Matrix:
[[938  68]
 [ 83 911]]


In [ ]:
wandb.log({"Accuracy": accuracy})
wandb.log({"classification_report": class_report})

In [ ]:
# Create a mapping of class names to integer indices
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Convert y_true and y_pred from strings to indices
y_true_int = [label_to_index[label] for label in y_true]
y_pred_int = [label_to_index[label] for label in y_pred]

# Log confusion matrix with the updated integer values
wandb.log({"confusion_matrix": wandb.plot.confusion_matrix(probs=None, y_true=y_true_int, preds=y_pred_int, class_names=labels)})

In [ ]:
wandb.finish()
model.config.use_cache = True

Accuracy,▁
eval/loss,█▃▃▂▁
eval/runtime,█▅▃▁▁
eval/samples_per_second,▁▄▆██
eval/steps_per_second,▁▅▇██
train/epoch,▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▇▇▇▇▇▇███
train/grad_norm,█▄▂▁▁▂▁▂▁▂▁▁▁▂▁▁▂▁▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂
train/learning_rate,█████▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁
train/loss,█▃▁▂▁▂▂▂▂▁▃▂▂▄▃▃▂▂▂▂▂▃▂▂▂▂▃▁▁▁▂▂▂▁▁▁▁▁▁▁
Accuracy,0.9245


# save model in Huging Face

In [ ]:
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
fine_tuned_model = "/content/llama-3.1-fine-tuned-model/"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch


# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, fine_tuned_model)
model = model.merge_and_unload()

In [ ]:
model_dir = f"Llama-31-8B-Instruct-suicide-related-text-classification_{run_date}"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2633: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

('Llama-31-8B-Instruct-suicide-related-text-classification_12-09-2024_15_17/tokenizer_config.json',
 'Llama-31-8B-Instruct-suicide-related-text-classification_12-09-2024_15_17/special_tokens_map.json',
 'Llama-31-8B-Instruct-suicide-related-text-classification_12-09-2024_15_17/tokenizer.json')

In [ ]:
model.push_to_hub(model_dir, use_temp_dir=False)
tokenizer.push_to_hub(model_dir, use_temp_dir=False)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2633: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AndresR2909/Llama-31-8B-Instruct-suicide-related-text-classification_12-09-2024_15_17/commit/2e895e03a3ac00791ad9a178a0ef2acf027da614', commit_message='Upload tokenizer', commit_description='', oid='2e895e03a3ac00791ad9a178a0ef2acf027da614', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
splits = {'train': 'data/train-00000-of-00001-7b34565378f02992.parquet', 'val': 'data/val-00000-of-00001-d7338c59b5e5031f.parquet', 'test': 'data/test-00000-of-00001-c830a979da438bff.parquet'}

df_val = pd.read_parquet("hf://datasets/PrevenIA/spanish-suicide-intent/" + splits["val"])

In [ ]:
df_val[df_val['Label'] == 1].head()

,Text,Label,dataset,__index_level_0__
9,Estoy en el mismo barco que tú. Mis padres son...,1,Hackaton Somos NLP,8929
14,En este punto ni siquiera me importa el dolor ...,1,Hackaton Somos NLP,712
24,Creo que necesito ayuda en mi situación actual...,1,Hackaton Somos NLP,2483
53,No sabes cuánto quería suicidarme.,1,Hackaton Somos NLP,4851
54,tan triste no puede dejar de pensar cómo suici...,1,Hackaton Somos NLP,3484


In [ ]:
text = df_val[df_val['Label'] == 1]["Text"].iloc[0]
prompt = f"""Clasifica el texto con la etiquta "1" si hay ideación/comportamiento suicida y la etiqueta "0" en otro caso, retorna la respuesta como la correspondiente etiqueta.
texto: {text}
etiqueta: """.strip()

prompt

'Clasifica el texto con la etiquta "1" si hay ideación/comportamiento suicida y la etiqueta "0" en otro caso, retorna la respuesta como la correspondiente etiqueta.\ntexto: Estoy en el mismo barco que tú. Mis padres son buena gente y me criaron bien y con amor, yo nací roto y defectuoso. Una vez que ambos se han ido, yo mismo me iré.\netiqueta:'

In [ ]:
model = f"/content/Llama-31-8B-Instruct-suicide-related-text-classification_{run_date}"

tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
        model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
outputs = pipe(prompt, max_new_tokens=2, do_sample=True, temperature=0.1)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(outputs[0]["generated_text"].split("etiqueta: ")[-1].strip())

1
